# Select the 50 repeat stations for the new network

## Description

This program reads the folium files (one record of each station, divided by groups of n occupattions) in order to do the following:
- To plot each occupation group over folium map
- The stations are then analyzed to choose which ones will be part of the 50 stations for the new network
- The underisable ones are removed from the analysis
- A folium file with the chosen stations is created

The priority order for the chosen is
1. Geographical coverage
2. Number of occupations
3. Values of RMSE

The details are written in the research journal

It produces these main maps:
1. Geographical coverage analysis: used to do the analysis and select the 50 repeat stations
2. Selected repeat stations and occupation groups: useful to see which kind of occupation group each selected repeat station was part of
3. Selected repeat stations vs unselected: useful to see which station made to the selection versus the ones who were eliminated
4. Map of the selected repeat stations its coverage radius active status toggable

In [1]:
# Import modules
import mestrado_module as mm
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium
from folium import plugins
import seaborn as sns
import branca
from branca.element import Template, MacroElement

In [2]:
# Magnetic Observatories data
mags_obs_folder: Path = Path(mm.path_00_data_manual)
mags_obs_file: Path = Path("models_intermagnet_observatories_south_america.csv")

# Input folder for folium files (REPEAT STATION DATA)
input_folder: Path = Path(mm.path_pipeline_05_rank_n_occupations)

# Files for Folium plot
folium_file_n_12: Path = Path(mm.output_5a_code_folium_file_n_12)
folium_file_n_10: Path = Path(mm.output_5a_code_folium_file_n_10)
folium_file_n_08: Path = Path(mm.output_5a_code_folium_file_n_08)
folium_file_n_06: Path = Path(mm.output_5a_code_folium_file_n_06)
folium_file_n_03: Path = Path(mm.output_5a_code_folium_file_n_03)
folium_file_n_01: Path = Path(mm.output_5a_code_folium_file_n_01)

# Modified files
mod_folium_file_n_12: Path = Path("modified_n_12_folium_plot.csv")
mod_folium_file_n_10: Path = Path("modified_n_10_folium_plot.csv")
mod_folium_file_n_08: Path = Path("modified_n_08_folium_plot.csv")
mod_folium_file_n_06: Path = Path("modified_n_06_folium_plot.csv")
mod_folium_file_n_03: Path = Path("modified_n_03_folium_plot.csv")
mod_folium_file_n_01: Path = Path("modified_n_01_folium_plot.csv")

# Save figures and files
output_folder: Path = Path(mm.path_pipeline_07_select_rs_geo_distribution)
chosen_rs: Path = Path(mm.output_7a_code_selected_rs_folium)

# Folium maps
mapa0 = "../02_pipeline/07_select_repeat_stations_geo_distribution/07a_selection_process_repeat_stations.html"
mapa1 = "../02_pipeline/07_select_repeat_stations_geo_distribution/07b_comparison_selected_repeat_stations_and_ocp_groups.html"
mapa2 = "../02_pipeline/07_select_repeat_stations_geo_distribution/07c_comparison_selected_repeat_stations_unselected_rs.html"
mapa3 = "../02_pipeline/07_select_repeat_stations_geo_distribution/07d_selected_repeat_stations_brazil_map.html"

## Read the data

In [3]:
# A dataframe for each n occupation group
df_n12 = pd.read_csv(input_folder/ folium_file_n_12)
#df_n12.info()

df_n10 = pd.read_csv(input_folder/ folium_file_n_10)
#df_n10.info()

df_n08 = pd.read_csv(input_folder/ folium_file_n_08)
#df_n08.info()

df_n06 = pd.read_csv(input_folder/ folium_file_n_06)
#df_n06.info()

df_n03 = pd.read_csv(input_folder/ folium_file_n_03)
#df_n03.info()

df_n01 = pd.read_csv(input_folder/ folium_file_n_01)
#df_n01.info()

## Original geodataframes, before station removal

In [4]:
## GDF FOR THE REPEAT STATION DATA
# n12
geometry_n12_old = [Point(xy) for xy in zip(df_n12["Lon_dd"], df_n12["Lat_dd"])]
# Create the geodataframe (use the df_aux dataframe and create the geometry column)
gdf_n12_old = gpd.GeoDataFrame(df_n12, geometry = geometry_n12_old)

# n10
geometry_n10_old = [Point(xy) for xy in zip(df_n10["Lon_dd"], df_n10["Lat_dd"])]
# Create the geodataframe (use the df_aux dataframe and create the geometry column)
gdf_n10_old = gpd.GeoDataFrame(df_n10, geometry = geometry_n10_old)

# n8
geometry_n08_old = [Point(xy) for xy in zip(df_n08["Lon_dd"], df_n08["Lat_dd"])]
# Create the geodataframe (use the df_aux dataframe and create the geometry column)
gdf_n08_old = gpd.GeoDataFrame(df_n08, geometry = geometry_n08_old)

# n6
geometry_n06_old = [Point(xy) for xy in zip(df_n06["Lon_dd"], df_n06["Lat_dd"])]
# Create the geodataframe (use the df_aux dataframe and create the geometry column)
gdf_n06_old = gpd.GeoDataFrame(df_n06, geometry = geometry_n06_old)

# n03
geometry_n03_old = [Point(xy) for xy in zip(df_n03["Lon_dd"], df_n03["Lat_dd"])]
# Create the geodataframe (use the df_aux dataframe and create the geometry column)
gdf_n03_old = gpd.GeoDataFrame(df_n03, geometry = geometry_n03_old)

# n01
geometry_n01_old = [Point(xy) for xy in zip(df_n01["Lon_dd"], df_n01["Lat_dd"])]
# Create the geodataframe (use the df_aux dataframe and create the geometry column)
gdf_n01_old = gpd.GeoDataFrame(df_n01, geometry = geometry_n01_old)

## Select repeat stations to be removed from the distribution

In [5]:
# Group n12
new_df_n12 = df_n12
new_df_n12.drop(new_df_n12[new_df_n12["Code"] == "GO_CAT"].index, inplace = True)
new_df_n12.drop(new_df_n12[new_df_n12["Code"] == "RN_NAT"].index, inplace = True)
new_df_n12.drop(new_df_n12[new_df_n12["Code"] == "MS_CGR"].index, inplace = True)
new_df_n12.drop(new_df_n12[new_df_n12["Code"] == "SP_MRL"].index, inplace = True)
new_df_n12.drop(new_df_n12[new_df_n12["Code"] == "RS_RGR"].index, inplace = True)
#new_df_n12.drop(new_df_n12[new_df_n12["Code"] == "GO_GOI"].index, inplace = True)
new_df_n12 = new_df_n12.reset_index()

# Check new df
#new_df_n12

In [6]:
# Group n10
new_df_n10 = df_n10
new_df_n10.drop(new_df_n10[new_df_n10["Code"] == "PI_PAR"].index, inplace = True)
new_df_n10.drop(new_df_n10[new_df_n10["Code"] == "PI_TER"].index, inplace = True)
new_df_n10.drop(new_df_n10[new_df_n10["Code"] == "PA_CAG"].index, inplace = True)
new_df_n10.drop(new_df_n10[new_df_n10["Code"] == "MG_DIV"].index, inplace = True)
new_df_n10.drop(new_df_n10[new_df_n10["Code"] == "SP_CTD"].index, inplace = True)
new_df_n10.drop(new_df_n10[new_df_n10["Code"] == "SP_IPV"].index, inplace = True)
new_df_n10.drop(new_df_n10[new_df_n10["Code"] == "PR_PAL"].index, inplace = True)
new_df_n10 = new_df_n10.reset_index()

# Check new df
#new_df_n10

In [7]:
# Group n08
new_df_n08 = df_n08
new_df_n08.drop(new_df_n08[new_df_n08["Code"] == "CE_CTO"].index, inplace = True)
new_df_n08.drop(new_df_n08[new_df_n08["Code"] == "AL_MCO"].index, inplace = True)
new_df_n08.drop(new_df_n08[new_df_n08["Code"] == "BA_MNV"].index, inplace = True)
new_df_n08.drop(new_df_n08[new_df_n08["Code"] == "BA_ILH"].index, inplace = True)
new_df_n08.drop(new_df_n08[new_df_n08["Code"] == "MG_DIA"].index, inplace = True)
new_df_n08.drop(new_df_n08[new_df_n08["Code"] == "MG_ITB"].index, inplace = True)
new_df_n08.drop(new_df_n08[new_df_n08["Code"] == "MT_CCR"].index, inplace = True)
new_df_n08.drop(new_df_n08[new_df_n08["Code"] == "PR_CBE"].index, inplace = True)
new_df_n08.drop(new_df_n08[new_df_n08["Code"] == "SP_SPO"].index, inplace = True)
new_df_n08.drop(new_df_n08[new_df_n08["Code"] == "SP_SJC"].index, inplace = True)
new_df_n08.drop(new_df_n08[new_df_n08["Code"] == "SC_FLO"].index, inplace = True)
#new_df_n08.drop(new_df_n08[new_df_n08["Code"] == "RS_SMA"].index, inplace = True)
new_df_n08.drop(new_df_n08[new_df_n08["Code"] == "RS_SBO"].index, inplace = True) # could te in place of RS_SMA
new_df_n08.drop(new_df_n08[new_df_n08["Code"] == "RS_URU"].index, inplace = True) # could te in place of RS_SMA
new_df_n08.drop(new_df_n08[new_df_n08["Code"] == "AM_TEF"].index, inplace = True)
new_df_n08.drop(new_df_n08[new_df_n08["Code"] == "TO_PNL"].index, inplace = True)
new_df_n08 = new_df_n08.reset_index()

# Check new df
#new_df_n08

In [8]:
# Group n06
new_df_n06 = df_n06
new_df_n06.drop(new_df_n06[new_df_n06["Code"] == "AP_AMP_B"].index, inplace = True)
new_df_n06.drop(new_df_n06[new_df_n06["Code"] == "MT_APG_A"].index, inplace = True)
new_df_n06.drop(new_df_n06[new_df_n06["Code"] == "BA_BJL"].index, inplace = True)
new_df_n06.drop(new_df_n06[new_df_n06["Code"] == "BA_BAR_A"].index, inplace = True)
new_df_n06.drop(new_df_n06[new_df_n06["Code"] == "MG_JAN"].index, inplace = True)
new_df_n06.drop(new_df_n06[new_df_n06["Code"] == "GO_BRA_A"].index, inplace = True)
new_df_n06.drop(new_df_n06[new_df_n06["Code"] == "MG_GVL"].index, inplace = True)
new_df_n06.drop(new_df_n06[new_df_n06["Code"] == "ES_VTO"].index, inplace = True)
new_df_n06.drop(new_df_n06[new_df_n06["Code"] == "MG_UBA"].index, inplace = True)
new_df_n06.drop(new_df_n06[new_df_n06["Code"] == "RJ_RJN"].index, inplace = True)
new_df_n06.drop(new_df_n06[new_df_n06["Code"] == "SP_BOT"].index, inplace = True)
new_df_n06.drop(new_df_n06[new_df_n06["Code"] == "SC_LAG"].index, inplace = True)
new_df_n06.drop(new_df_n06[new_df_n06["Code"] == "RS_PFD"].index, inplace = True)
new_df_n06.drop(new_df_n06[new_df_n06["Code"] == "RS_SAN"].index, inplace = True)
new_df_n06.drop(new_df_n06[new_df_n06["Code"] == "RS_BCH_A"].index, inplace = True)
new_df_n06 = new_df_n06.reset_index()

# Check new df
#new_df_n06

In [9]:
# Group n03
new_df_n03 = df_n03
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "AM_PRT"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "AM_TAB"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "PA_CCH_A"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "PA_ITA"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "RO_CMQ"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "RO_GMR"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "RO_FPB"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "BA_BRG_C"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "CE_BRB_A"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "PB_JPA"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "RN_MSS"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "SE_ARC_E"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "SE_ARC_F"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "GO_ARA_A"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "PR_PIT"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "RS_SVP"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "RS_BCH_B"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "RS_SMS"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "RS_ALE"].index, inplace = True) # could te in place of RS_SMA
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "GO_ARU_B"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "MG_PIR_D"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "MG_PIR_E"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "MG_ACI_A"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "SP_GPX"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "SP_TAU"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "MG_CBQ_A"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "MG_BHZ_C"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "MG_BHZ_E"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "MG_BBI_B"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "MG_ACI_B"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "BA_BRG_D"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "MT_RON"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "MS_PMT"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "MG_UBR_F"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "MG_BBI_D"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "MT_VBT_A"].index, inplace = True)
new_df_n03.drop(new_df_n03[new_df_n03["Code"] == "AM_ERP"].index, inplace = True)
new_df_n03 = new_df_n03.reset_index()

# Check new df
#new_df_n03

In [10]:
# Group n01
new_df_n01 = df_n01
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "RR_PCM_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "AM_JUT_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "AC_EPC_B"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "AM_COD_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "AM_MAU_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "PA_MDO"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "AP_AMP_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "PA_ALT"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "PA_ALT_C"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "PA_CPC_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MA_RAP"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MA_BCB_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "CE_ACU"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "RS_BAG_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "RS_MOS_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "RS_TRA_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "SC_CRI"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "RS_TPS_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "SC_ITA_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "SP_ITH_C"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "SP_ITH"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "SP_ATI_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "SP_CAM"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_PCA_B"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "RJ_SJB_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "RJ_CGZ"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "RJ_SPA"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_BBI_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_BBI_C"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_BHZ_G"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_BHZ_D"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_BHZ_F"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_BHZ_B"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_GVL_D"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_BHZ_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_DIA_D"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_PIR_C"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_ACI_C"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_JAN_E"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "GO_ARA"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "GO_ARA_B"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "GO_ARA_C"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_TIR"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_PMN"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_CUR"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_PIR_B"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "ES_CBR"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "BA_SCC"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "GO_ARU_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "GO_ARU_C"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_PIR_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "PA_VSU_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MA_CHA"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MA_BCD_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "PE_IGA_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "PE_SCC_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "AL_RLO_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "SE_ARC_C"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "BA_BRG_B"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "BA_VLC_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "BA_FST"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "SE_ARC_B"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "PE_ITA_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "PB_PAT"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "PI_PTP"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "PI_DAC"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "PI_BJP_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "PI_BJP_B"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MA_BLS_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MT_VRC_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MS_COX_A"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MT_CCR_B"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MT_PAN"].index, inplace = True) 
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "RO_PIM"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MT_GCN"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_MTC_D"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "BA_BAR_B"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "PA_ALT_B"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "AC_EPC"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MT_SIN"].index, inplace = True)
#new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MT_GUI"].index, inplace = True) #to be in place of GO_JAT excluded before
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "GO_FOR_B"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MG_CPL"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "TO_RAB"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "PR_LON"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "CE_CTE"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "RR_ROR_A"].index, inplace = True)
#new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "PA_PTR"].index, inplace = True)
new_df_n01.drop(new_df_n01[new_df_n01["Code"] == "MA_CAR_D"].index, inplace = True)
new_df_n01 = new_df_n01.reset_index()

# Check new df
#new_df_n01

In [11]:
# remove the following to get 50 repeat stations
# IN ORDER TO GET THE BEST COVERAGE WITH 50 STATIONS, SOME WITH HIGHER OCCUPATION RATE WILL BE REMOVED
# AM_TEF 9 OCP, TO_PNL 9 OCP

## Create a dataframe with the selected repeat stations

In [12]:
frames = [new_df_n12, new_df_n10, new_df_n08, new_df_n06, new_df_n03, new_df_n01]
selected_rs = pd.concat(frames)
selected_rs = selected_rs.drop(columns = ["index"])

# Save it
selected_rs.to_csv(output_folder / chosen_rs, index=False, float_format="%.3f", na_rep="NaN")

# Check it
selected_rs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 5
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   Code                      50 non-null     object  
 1   Lat_dd                    50 non-null     float64 
 2   Lon_dd                    50 non-null     float64 
 3   Alt_m                     50 non-null     float64 
 4   Time_dy                   50 non-null     float64 
 5   D_dd                      50 non-null     float64 
 6   IGRF_D_dd                 50 non-null     float64 
 7   I_dd                      50 non-null     float64 
 8   IGRF_I_dd                 50 non-null     float64 
 9   F_nT                      50 non-null     float64 
 10  IGRF_F_nT                 50 non-null     float64 
 11  H_nT                      50 non-null     float64 
 12  IGRF_H_nT                 50 non-null     float64 
 13  X_nT                      50 non-null     float64 
 1

## Create the geodataframes for the geographical folium plots using the selected stations

In [13]:
## GDF FOR THE REPEAT STATION DATA
# n12
geometry_n12 = [Point(xy) for xy in zip(new_df_n12["Lon_dd"], new_df_n12["Lat_dd"])]
# Create the geodataframe (use the df_aux dataframe and create the geometry column)
gdf_n12 = gpd.GeoDataFrame(new_df_n12, geometry = geometry_n12)

# n10
geometry_n10 = [Point(xy) for xy in zip(new_df_n10["Lon_dd"], new_df_n10["Lat_dd"])]
# Create the geodataframe (use the df_aux dataframe and create the geometry column)
gdf_n10 = gpd.GeoDataFrame(new_df_n10, geometry = geometry_n10)

# n8
geometry_n08 = [Point(xy) for xy in zip(new_df_n08["Lon_dd"], new_df_n08["Lat_dd"])]
# Create the geodataframe (use the df_aux dataframe and create the geometry column)
gdf_n08 = gpd.GeoDataFrame(new_df_n08, geometry = geometry_n08)

# n6
geometry_n06 = [Point(xy) for xy in zip(new_df_n06["Lon_dd"], new_df_n06["Lat_dd"])]
# Create the geodataframe (use the df_aux dataframe and create the geometry column)
gdf_n06 = gpd.GeoDataFrame(new_df_n06, geometry = geometry_n06)

# n03
geometry_n03 = [Point(xy) for xy in zip(new_df_n03["Lon_dd"], new_df_n03["Lat_dd"])]
# Create the geodataframe (use the df_aux dataframe and create the geometry column)
gdf_n03 = gpd.GeoDataFrame(new_df_n03, geometry = geometry_n03)

# n01
geometry_n01 = [Point(xy) for xy in zip(new_df_n01["Lon_dd"], new_df_n01["Lat_dd"])]
# Create the geodataframe (use the df_aux dataframe and create the geometry column)
gdf_n01 = gpd.GeoDataFrame(new_df_n01, geometry = geometry_n01)

geometry_selected_rs = [Point(xy) for xy in zip(selected_rs["Lon_dd"], selected_rs["Lat_dd"])]
# Create the geodataframe (use the df_aux dataframe and create the geometry column)
gdf_selected_rs = gpd.GeoDataFrame(selected_rs, geometry = geometry_selected_rs)

In [14]:
## GDF FOR THE MAGNETIC OBSERVATORIES
# Read magnetic observatory data file
obs_data_df = pd.read_csv(mags_obs_folder / mags_obs_file)
# Check df info
obs_data_df.info()

# Create the geometry for the geodataframe
obs_data_geometry = [Point(xy) for xy in zip(obs_data_df["Lon_dd"], obs_data_df["Lat_dd"])]

# Create the geodataframe
gdf_obs_data = gpd.GeoDataFrame(obs_data_df, geometry = obs_data_geometry)
gdf_obs_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Code        6 non-null      object 
 1   Name        6 non-null      object 
 2   Lat_dd      6 non-null      float64
 3   Lon_dd      6 non-null      float64
 4   Altitude_m  6 non-null      int64  
 5   Country     6 non-null      object 
dtypes: float64(2), int64(1), object(3)
memory usage: 416.0+ bytes
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Code        6 non-null      object  
 1   Name        6 non-null      object  
 2   Lat_dd      6 non-null      float64 
 3   Lon_dd      6 non-null      float64 
 4   Altitude_m  6 non-null      int64   
 5   Country     6 non-null      object  
 6   geometry    6 non-null      geometry
dtypes: float64(2),

## Plot maps using folium

In [15]:
## DEFINITIONS (distances in km)
#observatory_coverage_radius = 904000 # BASED ON ITALY CASTELLO TESINO
#embracer_coverage_radius = 150000 # germany
#embracer_coverage_radius = 230000 # italy GIB station

# Radius suggested by Katia and Cristiano (800 km, 500 km, 300 km, 500 km)
observatory_coverage_radius_m = 800000
repeat_station_coverage_radius_m = 300000

# Color definitions
n12_color = "red"
n10_color = "orange"
n08_color = "lightgreen"
n06_color = "green"
n03_color = "blue"
n01_color = "purple"
obs_color = "darkblue"
selected_rs_color = "black"

### Map for the selection process using the established criteria

In [16]:
# INTERACTIVE MAP: it was used for the selection process

# Create the Interactive map using folium
tile_type0 = "OpenStreetMap"
map0 = folium.Map(location = [-15, -50], zoom_start = 5, tiles = tile_type0)

# Main group
fg0 = folium.FeatureGroup()    

# Group 1: Repeat Stations with 12 or more occupations
g01 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Occupations: 12 or more', show = True)
for index, location_info in gdf_n12.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = n12_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["RS_name"],
            "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
            "Last occupation time:",
            location_info["Time_dy"],
        ],
    ).add_to(g01)
# Add the repeat stations coverage    
for index, location_info in gdf_n12.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = repeat_station_coverage_radius_m,
        color = n12_color #,
        #fill_color = "green"    
    ).add_to(g01)


# Group 2: Repeat Stations with 10 to 11 occupations
g02 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Occupations: 10 to 11', show = False)
for index, location_info in gdf_n10.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = n10_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
             "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
            "Last occupation time:",
            location_info["Time_dy"],
        ],
    ).add_to(g02)
# Add the repeat stations coverage    
for index, location_info in gdf_n10.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = repeat_station_coverage_radius_m,
        color = n10_color #,
        #fill_color = "green"    
    ).add_to(g02)


# Group 3: Repeat Stations with 8 to 9 occupations
g03 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Occupations: 8 to 9', show = False)
for index, location_info in gdf_n08.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = n08_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
             "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
            "Last occupation time:",
            location_info["Time_dy"],
        ],
    ).add_to(g03)
# Add the repeat stations coverage    
for index, location_info in gdf_n08.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = repeat_station_coverage_radius_m,
        color = n08_color #,
        #fill_color = "green"    
    ).add_to(g03)

    
# Group 4: Repeat Stations with 6 to 7 occupations
g04 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Occupations: 6 to 7', show = False)
for index, location_info in gdf_n06.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = n06_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
             "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
            "Last occupation time:",
            location_info["Time_dy"],
        ],
    ).add_to(g04)
# Add the repeat stations coverage    
for index, location_info in gdf_n06.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = repeat_station_coverage_radius_m,
        color = n06_color #,
        #fill_color = "green"    
    ).add_to(g04)

    
# Group 5: Repeat Stations with 3 to 5 occupations
g05 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Occupations: 3 to 5', show = False)
for index, location_info in gdf_n03.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = n03_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
             "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
            "Last occupation time:",
            location_info["Time_dy"],
        ],
    ).add_to(g05)
# Add the repeat stations coverage    
for index, location_info in gdf_n03.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = repeat_station_coverage_radius_m,
        color = n03_color #,
        #fill_color = "green"    
    ).add_to(g05)
    
    
# Group 6: Repeat Stations with 1 to 2 occupations
g06 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Occupations: 1 to 2', show = False)
for index, location_info in gdf_n01.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = n01_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
             "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
            "Last occupation time:",
            location_info["Time_dy"],
        ],
    ).add_to(g06)
# Add the repeat stations coverage    
for index, location_info in gdf_n01.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = repeat_station_coverage_radius_m,
        color = n01_color #,
        #fill_color = "green"    
    ).add_to(g06)    
    

# Group 7: Magnetic Observatories in South America
g07 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Magnetic Observatories South America', show = False)
for index, location_info in gdf_obs_data.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = obs_color, icon ="square", prefix = "fa"),
        tooltip = [
            "Observatory:",
            location_info["Name"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["Country"],
        ],
    ).add_to(g07)
# Add the observatory coverage    
for index, location_info in gdf_obs_data.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = observatory_coverage_radius_m,
        color = obs_color,
        fill_color = obs_color
    ).add_to(g07)
        
# Add the subgroups to the main map
map0.add_child(fg0)
map0.add_child(g01)
map0.add_child(g02)
map0.add_child(g03)
map0.add_child(g04)
map0.add_child(g05)
map0.add_child(g06)
map0.add_child(g07)

# Add other layers to the map
folium.TileLayer('Stamen Terrain').add_to(map0)
folium.TileLayer('Stamen Toner').add_to(map0)
#folium.TileLayer('Stamen Water Color').add_to(map0)
folium.TileLayer('cartodbpositron').add_to(map0)
#folium.TileLayer('cartodbdark_matter').add_to(map0)

# Add the layer control
folium.LayerControl().add_to(map0)

# Add altitude and longitude tool map
lat_lon_vis0 = folium.LatLngPopup()
map0.add_child(lat_lon_vis0)

# Measurement control
measure_control0 = plugins.MeasureControl(position = "topleft", 
                                         active_color = "red", 
                                         completed_color = "red", 
                                         primary_length_unit = "kilometers")
map0.add_child(measure_control0)

# Add the full screen button
fullscreen_button0 = plugins.Fullscreen(position ='topright', title ='Expand me',
                                title_cancel ='Exit me',
                                force_separate_button = True)
map0.add_child(fullscreen_button0)

# Add a mini map
minimap0 = plugins.MiniMap()
map0.add_child(minimap0)

# Draw tools
# export=True exports the drawn shapes as a geojson file
draw0 = plugins.Draw(export=True)
map0.add_child(draw0)


# ADD LEGEND TO THE MAP
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Selecting 50 repeat stations to cover the territory</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend (draggable!)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:red;opacity:0.7;'></span>Occupations: 12 or more</li>
    <li><span style='background:orange;opacity:0.7;'></span>Occupations: 10 to 11</li>
    <li><span style='background:lightgreen;opacity:0.7;'></span>Occupations: 8 to 9</li>
    <li><span style='background:green;opacity:0.7;'></span>Occupations: 6 to 7</li>
    <li><span style='background:blue;opacity:0.7;'></span>Occupations: 3 to 5</li>
    <li><span style='background:purple;opacity:0.7;'></span>Occupations: 1 to 2</li>
    <li><span style='background:darkblue;opacity:0.7;'></span>Magnetic Observatories</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)
map0.get_root().add_child(macro)


# Save map
map0.save(mapa0)

### Map showing the comparison between the selected repeat stations and their occupation groups

In [17]:
# INTERACTIVE MAP: comparison between the selected stations and ther markers showing their number of occupations (this only contains the selected repeat station in each n group)
# useful to see which kind of occupation group each selected repeat station was part of

# Create the Interactive map using folium
tile_type1 = "OpenStreetMap"
map1 = folium.Map(location = [-15, -50], zoom_start = 5, tiles = tile_type1)

# Main group
fg0 = folium.FeatureGroup()    

# Group 1: Repeat Stations with 12 or more occupations
g01 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Occupations: 12 or more', show = False)
for index, location_info in gdf_n12.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = n12_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["RS_name"],
            "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
            "Last occupation time:",
            location_info["Time_dy"],
        ],
    ).add_to(g01)
# Add the repeat stations coverage    
for index, location_info in gdf_n12.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = repeat_station_coverage_radius_m,
        color = n12_color #,
        #fill_color = "green"    
    ).add_to(g01)


# Group 2: Repeat Stations with 10 to 11 occupations
g02 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Occupations: 10 to 11', show = False)
for index, location_info in gdf_n10.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = n10_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
             "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
            "Last occupation time:",
            location_info["Time_dy"],
        ],
    ).add_to(g02)
# Add the repeat stations coverage    
for index, location_info in gdf_n10.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = repeat_station_coverage_radius_m,
        color = n10_color #,
        #fill_color = "green"    
    ).add_to(g02)


# Group 3: Repeat Stations with 8 to 9 occupations
g03 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Occupations: 8 to 9', show = False)
for index, location_info in df_n08.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = n08_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
             "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
        ],
    ).add_to(g03)
# Add the repeat stations coverage    
for index, location_info in gdf_n08.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = repeat_station_coverage_radius_m,
        color = n08_color #,
        #fill_color = "green"    
    ).add_to(g03)

    
# Group 4: Repeat Stations with 6 to 7 occupations
g04 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Occupations: 6 to 7', show = False)
for index, location_info in gdf_n06.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = n06_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
             "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
        ],
    ).add_to(g04)
# Add the repeat stations coverage    
for index, location_info in gdf_n06.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = repeat_station_coverage_radius_m,
        color = n06_color #,
        #fill_color = "green"    
    ).add_to(g04)

    
# Group 5: Repeat Stations with 3 to 5 occupations
g05 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Occupations: 3 to 5', show = False)
for index, location_info in gdf_n03.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = n03_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
             "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
        ],
    ).add_to(g05)
# Add the repeat stations coverage    
for index, location_info in gdf_n03.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = repeat_station_coverage_radius_m,
        color = n03_color #,
        #fill_color = "green"    
    ).add_to(g05)
    
    
# Group 6: Repeat Stations with 1 to 2 occupations
g06 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Occupations: 1 to 2', show = False)
for index, location_info in gdf_n01.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = n01_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
             "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
        ],
    ).add_to(g06)
# Add the repeat stations coverage    
for index, location_info in gdf_n01.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = repeat_station_coverage_radius_m,
        color = n01_color #,
        #fill_color = "green"    
    ).add_to(g06)    
    

# Group 7: Magnetic Observatories in South America
g07 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Magnetic Observatories South America', show = False)
for index, location_info in gdf_obs_data.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = obs_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Observatory:",
            location_info["Name"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["Country"]
        ],
    ).add_to(g07)
# Add the observatory coverage    
for index, location_info in gdf_obs_data.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = observatory_coverage_radius_m,
        color = obs_color,
       fill_color = obs_color
    ).add_to(g07)

    
# Group 8: Selected 50 repeat station
g08 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'The 50 selected repeat stations', show = True)
for index, location_info in gdf_selected_rs.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = selected_rs_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
             "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
             "Last occupation time:",
            location_info["Time_dy"],
        ],
    ).add_to(g08)
# Add the observatory coverage    
for index, location_info in gdf_selected_rs.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = repeat_station_coverage_radius_m,
        color = selected_rs_color,
      #  fill_color = obs_color
    ).add_to(g08)    
    
     
# Add the subgroups to the main map
map1.add_child(fg0)
map1.add_child(g01)
map1.add_child(g02)
map1.add_child(g03)
map1.add_child(g04)
map1.add_child(g05)
map1.add_child(g06)
map1.add_child(g07)
map1.add_child(g08)

# Add other layers to the map
folium.TileLayer('Stamen Terrain').add_to(map1)
folium.TileLayer('Stamen Toner').add_to(map1)
#folium.TileLayer('Stamen Water Color').add_to(map1)
folium.TileLayer('cartodbpositron').add_to(map1)
#folium.TileLayer('cartodbdark_matter').add_to(map1)

# Add the layer control
folium.LayerControl().add_to(map1)

# Add altitude and longitude tool map
lat_lon_vis0 = folium.LatLngPopup()
map1.add_child(lat_lon_vis0)

# Measurement control
measure_control0 = plugins.MeasureControl(position = "topleft", 
                                         active_color = "red", 
                                         completed_color = "red", 
                                         primary_length_unit = "kilometers")
map1.add_child(measure_control0)

# Add the full screen button
fullscreen_button0 = plugins.Fullscreen(position ='topright', title ='Expand me',
                                title_cancel ='Exit me',
                                force_separate_button = True)
map1.add_child(fullscreen_button0)

# Add a mini map
minimap1 = plugins.MiniMap()
map1.add_child(minimap1)

# Draw tools
# export=True exports the drawn shapes as a geojson file
draw0 = plugins.Draw(export=True)
map1.add_child(draw0)


# ADD LEGEND TO THE MAP
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>The selected 50 repeat stations and each n occupation group comparison</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend (draggable!)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:red;opacity:0.7;'></span>Occupations: 12 or more</li>
    <li><span style='background:orange;opacity:0.7;'></span>Occupations: 10 to 11</li>
    <li><span style='background:lightgreen;opacity:0.7;'></span>Occupations: 8 to 9</li>
    <li><span style='background:green;opacity:0.7;'></span>Occupations: 6 to 7</li>
    <li><span style='background:blue;opacity:0.7;'></span>Occupations: 3 to 5</li>
    <li><span style='background:purple;opacity:0.7;'></span>Occupations: 1 to 2</li>
    <li><span style='background:darkblue;opacity:0.7;'></span>Magnetic Observatories</li>
    <li><span style='background:black;opacity:0.7;'></span>Selected Repeat Stations</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)
map1.get_root().add_child(macro)


# Save map
map1.save(mapa1)

### Map of comparison between the selected and unselected stations

In [18]:
# INTERACTIVE MAP
# Selected repeat stations vs unselected: useful to see which station made to the selection versus the ones who were eliminated

# Create the Interactive map using folium
tile_type2 = "OpenStreetMap"
map2 = folium.Map(location = [-15, -50], zoom_start = 5, tiles = tile_type2)

# Main group
fg0 = folium.FeatureGroup()    

# Group 1: Repeat Stations with 12 or more occupations (no station removal in this group)
g01 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Occupations: 12 or more', show = False)
for index, location_info in gdf_n12_old.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = n12_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["RS_name"],
            "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
            "Last occupation time:",
            location_info["Time_dy"],
        ],
    ).add_to(g01)
# Add the repeat stations coverage    
#for index, location_info in gdf_n12.iterrows():    
#    folium.Circle(
#        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        radius = repeat_station_coverage_radius_m,
#        color = n12_color #,
#        #fill_color = "green"    
#    ).add_to(g01)


# Group 2: Repeat Stations with 10 to 11 occupations (no station removal in this group)
g02 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Occupations: 10 to 11', show = False)
for index, location_info in gdf_n10_old.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = n10_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
             "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
            "Last occupation time:",
            location_info["Time_dy"],
        ],
    ).add_to(g02)
# Add the repeat stations coverage    
#for index, location_info in gdf_n10.iterrows():    
#    folium.Circle(
#        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        radius = repeat_station_coverage_radius_m,
#        color = n10_color #,
#        #fill_color = "green"    
#    ).add_to(g02)


# Group 3: Repeat Stations with 8 to 9 occupations (no station removal in this group)
g03 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Occupations: 8 to 9', show = False)
for index, location_info in gdf_n08_old.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = n08_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
             "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
            "Last occupation time:",
            location_info["Time_dy"],
        ],
    ).add_to(g03)
# Add the repeat stations coverage    
#for index, location_info in gdf_n08.iterrows():    
#    folium.Circle(
#        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        radius = repeat_station_coverage_radius_m,
#        color = n08_color #,
#        #fill_color = "green"    
#    ).add_to(g03)

    
# Group 4: Repeat Stations with 6 to 7 occupations (no station removal in this group)
g04 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Occupations: 6 to 7', show = False)
for index, location_info in gdf_n06_old.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = n06_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
             "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
            "Last occupation time:",
            location_info["Time_dy"],
        ],
    ).add_to(g04)
# Add the repeat stations coverage    
#for index, location_info in gdf_n06.iterrows():    
#    folium.Circle(
#        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        radius = repeat_station_coverage_radius_m,
#        color = n06_color #,
#        #fill_color = "green"    
#    ).add_to(g04)

    
# Group 5: Repeat Stations with 3 to 5 occupations (no station removal in this group)
g05 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Occupations: 3 to 5', show = False)
for index, location_info in gdf_n03_old.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = n03_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
             "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
            "Last occupation time:",
            location_info["Time_dy"],
        ],
    ).add_to(g05)
# Add the repeat stations coverage    
#for index, location_info in gdf_n03.iterrows():    
#    folium.Circle(
#        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        radius = repeat_station_coverage_radius_m,
#        color = n03_color #,
#        #fill_color = "green"    
#    ).add_to(g05)
    
    
# Group 6: Repeat Stations with 1 to 2 occupations (no station removal in this group)
g06 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Occupations: 1 to 2', show = False)
for index, location_info in gdf_n01_old.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = n01_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
             "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
            "Last occupation time:",
            location_info["Time_dy"],
        ],
    ).add_to(g06)
# Add the repeat stations coverage    
#for index, location_info in gdf_n01.iterrows():    
#    folium.Circle(
#        [location_info["Lat_dd"], location_info["Lon_dd"]],
#        radius = repeat_station_coverage_radius_m,
#        color = n01_color #,
#        #fill_color = "green"    
#    ).add_to(g06)    
    

# Group 7: Magnetic Observatories in South America
g07 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Magnetic Observatories South America', show = False)
for index, location_info in gdf_obs_data.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = obs_color, icon ="square", prefix = "fa"),
        tooltip = [
            "Observatory:",
            location_info["Name"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
            location_info["Country"]
        ],
    ).add_to(g07)
# Add the observatory coverage    
for index, location_info in gdf_obs_data.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = observatory_coverage_radius_m,
        color = obs_color,
        fill_color = obs_color
    ).add_to(g07)
        

# Group 8: Selected 50 repeat station
g08 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'The 50 selected repeat stations', show = True)
for index, location_info in gdf_selected_rs.iterrows():
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        icon = folium.Icon(color = selected_rs_color, icon ="circle", prefix = "fa"),
        tooltip = [
            "Repeat_Station:",
            location_info["Code"],
            "Lat_dd:",
            location_info["Lat_dd"],
            "Lon_dd",
            location_info["Lon_dd"],
            "Local:",
             "Closest Brazilian Observatory:",
            location_info["Closest_OBS"],
            "Number of occupations:",
            location_info["N_occupations"],
            "Last occupation time:",
            location_info["Time_dy"],
        ],
    ).add_to(g08)
# Add the observatory coverage    
for index, location_info in gdf_selected_rs.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = repeat_station_coverage_radius_m,
        color = selected_rs_color,
      #  fill_color = obs_color
    ).add_to(g08)  
        
        
        
# Add the subgroups to the main map
map2.add_child(fg0)
map2.add_child(g01)
map2.add_child(g02)
map2.add_child(g03)
map2.add_child(g04)
map2.add_child(g05)
map2.add_child(g06)
map2.add_child(g07)
map2.add_child(g08)

# Add other layers to the map
folium.TileLayer('Stamen Terrain').add_to(map2)
folium.TileLayer('Stamen Toner').add_to(map2)
#folium.TileLayer('Stamen Water Color').add_to(map2)
folium.TileLayer('cartodbpositron').add_to(map2)
#folium.TileLayer('cartodbdark_matter').add_to(map2)

# Add the layer control
folium.LayerControl().add_to(map2)

# Add altitude and longitude tool map
lat_lon_vis0 = folium.LatLngPopup()
map2.add_child(lat_lon_vis0)

# Measurement control
measure_control0 = plugins.MeasureControl(position = "topleft", 
                                         active_color = "red", 
                                         completed_color = "red", 
                                         primary_length_unit = "kilometers")
map2.add_child(measure_control0)

# Add the full screen button
fullscreen_button0 = plugins.Fullscreen(position ='topright', title ='Expand me',
                                title_cancel ='Exit me',
                                force_separate_button = True)
map2.add_child(fullscreen_button0)

# Add a mini map
minimap2 = plugins.MiniMap()
map2.add_child(minimap2)

# Draw tools
# export=True exports the drawn shapes as a geojson file
draw0 = plugins.Draw(export=True)
map2.add_child(draw0)


# ADD LEGEND TO THE MAP
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Comparison between the selected 50 repeat stations versus the unselected ones</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend (draggable!)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:red;opacity:0.7;'></span>Occupations: 12 or more</li>
    <li><span style='background:orange;opacity:0.7;'></span>Occupations: 10 to 11</li>
    <li><span style='background:lightgreen;opacity:0.7;'></span>Occupations: 8 to 9</li>
    <li><span style='background:green;opacity:0.7;'></span>Occupations: 6 to 7</li>
    <li><span style='background:blue;opacity:0.7;'></span>Occupations: 3 to 5</li>
    <li><span style='background:purple;opacity:0.7;'></span>Occupations: 1 to 2</li>
    <li><span style='background:darkblue;opacity:0.7;'></span>Magnetic Observatories</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)
map2.get_root().add_child(macro)


# Save map
map2.save(mapa2)

### New Brazilian Repeat Station Network Map: geographical coverage

In [19]:
# Color definitions
sa_obs_color = "darkblue"
selected_rs_color = "black"

# Custom icons for the repeat stations and observatories
icon_path_sa_obs = r"../00_data/manual/symbols_square_darkblue.png"
icon_path_rs = r"../00_data/manual/symbols_circle_black.png"

In [20]:
# Fuction to create popup with Magnetic Observatory information
def popup_html_observatory(row):
    i = row
    code= gdf_obs_data['Code'].iloc[i] 
    obs_name= gdf_obs_data['Name'].iloc[i]
    lat = gdf_obs_data['Lat_dd'].iloc[i] 
    lon= gdf_obs_data['Lon_dd'].iloc[i] 
    altitude = gdf_obs_data['Altitude_m'].iloc[i]                     
    country = gdf_obs_data['Country'].iloc[i]

    left_col_color = "#19a7bd"
    right_col_color = "#f2f0d3"
    
    html_obs = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:10"; width="200px">{}</h4>""".format(code) + """

</head>
    <table style="height: 126px; width: 350px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Observatory Name</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(obs_name) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Latitude (dd)</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(lat) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Longitude (dd)</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(lon) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Altitude (m)</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(altitude) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Country</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(country) + """
</tr>
</tbody>
</table>
</html>
"""
    return html_obs

In [21]:
# Fuction to create popup with repeat station information
def popup_html(row):
    i = row
    code= gdf_selected_rs['Code'].iloc[i] 
    rs_name= gdf_selected_rs['RS_name'].iloc[i]
    lat = gdf_selected_rs['Lat_dd'].iloc[i] 
    lon= gdf_selected_rs['Lon_dd'].iloc[i] 
    n_ocp = gdf_selected_rs['N_occupations'].iloc[i]                     
    last_ocp = gdf_selected_rs['Time_dy'].iloc[i]
    closest_obs = gdf_selected_rs['Closest_OBS'].iloc[i]

    left_col_color = "#19a7bd"
    right_col_color = "#f2f0d3"
    
    html = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:10"; width="200px">{}</h4>""".format(code) + """

</head>
    <table style="height: 126px; width: 350px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Name</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(rs_name) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Latitude (dd)</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(lat) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Longitude (dd)</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(lon) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Total occupations</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(n_ocp) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Last occupation</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(last_ocp) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Nearest Brazilian Observatory</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(closest_obs) + """
</tr>
</tbody>
</table>
</html>
"""
    return html

In [22]:
# INTERACTIVE MAP
# Map of the selected repeat stations its coverage radius active status toggable

# Create the Interactive map using folium
tile_type3 = "OpenStreetMap"
map3 = folium.Map(location = [-15, -50], zoom_start = 5, tiles = tile_type3)

# Main group
fg0 = folium.FeatureGroup()    

# Group 1: Magnetic Observatories in South America
g01 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Magnetic Observatories South America, inactive radius', show = True)
for index, location_info in gdf_obs_data.iterrows():
    html_observatory = popup_html_observatory(index)
    iframe = branca.element.IFrame(html=html_observatory,width=510,height=280)
    popup = folium.Popup(folium.Html(html_observatory, script=True), max_width=500) 
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        popup=popup, 
        icon = folium.features.CustomIcon(icon_path_sa_obs, icon_size=(14, 14)),
        tooltip = "Click me",
    ).add_to(g01)
    

# Group 2: Magnetic Observatories in South America
g02 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'Magnetic Observatories South America, active radius', show = False)
for index, location_info in gdf_obs_data.iterrows():
    html_observatory = popup_html_observatory(index)
    iframe = branca.element.IFrame(html=html_observatory,width=510,height=280)
    popup = folium.Popup(folium.Html(html_observatory, script=True), max_width=500) 
    folium.Marker(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        popup=popup, 
        icon = folium.features.CustomIcon(icon_path_sa_obs, icon_size=(14, 14)),
        tooltip = "Click me",
    ).add_to(g02)
# Add the observatory coverage    
for index, location_info in gdf_obs_data.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = observatory_coverage_radius_m,
        color = sa_obs_color,
        fill_color = sa_obs_color
    ).add_to(g02)       
        
        
# Group 3: Selected 50 repeat station, active radius
g03 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'The 50 selected repeat stations, inactive radius', show = True)
for i in range(0,len(gdf_selected_rs)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html,width=510,height=280)
    popup = folium.Popup(folium.Html(html, script=True), max_width=500)
    folium.Marker([gdf_selected_rs['Lat_dd'].iloc[i], gdf_selected_rs['Lon_dd'].iloc[i]],
                popup=popup,icon=folium.features.CustomIcon(icon_path_rs, icon_size=(14, 14)),tooltip="Click me").add_to(g03)


# Group 4: Selected 50 repeat station, inactive radius
g04 = folium.plugins.FeatureGroupSubGroup(fg0, name = 'The 50 selected repeat stations, active radius', show = False)
for i in range(0,len(gdf_selected_rs)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html,width=510,height=280)
    popup = folium.Popup(folium.Html(html, script=True), max_width=500)
    folium.Marker([gdf_selected_rs['Lat_dd'].iloc[i], gdf_selected_rs['Lon_dd'].iloc[i]],
                popup=popup,icon=folium.features.CustomIcon(icon_path_rs, icon_size=(14, 14)),tooltip="Click me").add_to(g04)

# Add the observatory coverage    
for index, location_info in gdf_selected_rs.iterrows():    
    folium.Circle(
        [location_info["Lat_dd"], location_info["Lon_dd"]],
        radius = repeat_station_coverage_radius_m,
        color = selected_rs_color,
      #  fill_color = obs_color
    ).add_to(g04)
    

# Add the subgroups to the main map
map3.add_child(fg0)
map3.add_child(g01)
map3.add_child(g02)
map3.add_child(g03)
map3.add_child(g04)

# Add other layers to the map
folium.TileLayer('Stamen Terrain').add_to(map3)
folium.TileLayer('Stamen Toner').add_to(map3)
#folium.TileLayer('Stamen Water Color').add_to(map3)
folium.TileLayer('cartodbpositron').add_to(map3)
#folium.TileLayer('cartodbdark_matter').add_to(map3)

# Add the layer control
folium.LayerControl().add_to(map3)

# Add altitude and longitude tool map
lat_lon_vis0 = folium.LatLngPopup()
map3.add_child(lat_lon_vis0)

# Measurement control
measure_control0 = plugins.MeasureControl(position = "topleft", 
                                         active_color = "red", 
                                         completed_color = "red", 
                                         primary_length_unit = "kilometers")
map3.add_child(measure_control0)

# Add the full screen button
fullscreen_button0 = plugins.Fullscreen(position ='topright', title ='Expand me',
                                title_cancel ='Exit me',
                                force_separate_button = True)
map3.add_child(fullscreen_button0)

# Add a mini map
minimap3 = plugins.MiniMap()
map3.add_child(minimap3)

# Draw tools
# export=True exports the drawn shapes as a geojson file
draw0 = plugins.Draw(export=True)
map3.add_child(draw0)


#--------------------------------------------------------------
# ADD LEGEND TO THE MAP

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>New Brazilian Repeat Station Network: geographical coverage</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend (draggable!)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:black;opacity:0.7;'></span>Selected Repeat Stations</li>
    <li><span style='background:darkblue;opacity:0.7;'></span>Magnetic Observatories</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)
map3.get_root().add_child(macro)


# Save map
map3.save(mapa3)